In [ ]:
import sys
sys.path.append('../train')

import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
import cv2
import threading
import time

from learn import SimpleCNN,CustomResNet,CustomCircleDataset
from jetbot import Robot, Camera, bgr8_to_jpeg
import Jetson.GPIO as GPIO

## カメラ

In [ ]:
camera = Camera.instance()



## GUI

In [ ]:
widget_width = 224
widget_height = 224

camera_widget = widgets.Image(format='jpg', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpg', width=widget_width, height=widget_height)
image_layout = widgets.HBox([camera_widget])

#mask slider
low_h_slider = widgets.IntSlider(description='low h', min=0, max=179, value=90,step=1)
high_h_slider = widgets.IntSlider(description=' high h', min=0, max=179, value=100,step=1)
low_s_slider = widgets.IntSlider(description=' low s', min=0, max=255, value=140,step=1)
high_s_slider = widgets.IntSlider(description=' high s', min=0, max=255, value=255,step=1)
low_v_slider = widgets.IntSlider(description=' low v', min=0, max=255, value=0,step=1)
high_v_slider = widgets.IntSlider(description=' high v', min=0, max=255, value=255,step=1)

h_slider = widgets.HBox([low_h_slider, high_h_slider])
s_slider = widgets.HBox([low_s_slider, high_s_slider])
v_slider = widgets.HBox([low_v_slider, high_v_slider])
slider = widgets.VBox([h_slider,s_slider,v_slider])

#robot param slider
far_slider = widgets.IntSlider(description='far rad', min=0, max=100, value=20,step=1)
near_slider = widgets.IntSlider(description=' near rad', min=0, max=100, value=30,step=1)
speed_slider = widgets.FloatSlider(description='speed', min=0, max=1.0, value=0,step=0.1)
interval_slider = widgets.FloatSlider(description=' interval', min=0, max=5.0, value=0.5,step=0.1)
robot_slider = widgets.VBox([far_slider, near_slider, speed_slider, interval_slider])

layout = widgets.Layout(width='128px', height='64px')
snap_button = widgets.Button(description='snapshot', button_style='success', layout=layout)
percent = widgets.IntText(layout = layout,value = 0)
gui_layout =  widgets.HBox([image_layout,percent])

## モデル

In [ ]:
# デバイスの設定（CUDAが利用可能な場合はGPUを使用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルのインスタンスを作成
model = SimpleCNN()
# モデルの重みをロードし、推論モードに設定
model.load_state_dict(torch.load('../train/weight/simpleCNN_v3.pth'))
model.to(device)  # モデルをGPUに移動
model.eval()

## グローバル変数

In [ ]:
x, y, r = 0, 0, 0
#マルチスレッドで競合しないようにロック
lock = threading.Lock()

## 関数定義

In [ ]:
def model_input(image,model,device):
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    img_tonsor = transform(pil_image).unsqueeze(0).to(device)  # テンソルをGPUに移動
    
    with torch.no_grad():
            outputs = model(img_tonsor)
            predicted_coords = outputs.cpu().numpy()[0]  # 結果をCPUに戻す
            pos_x, pos_y, pos_r = predicted_coords
            return pos_x, pos_y, pos_r
        

#マスク処理する関数
def apply_hsv_threshold(image):
#     low_h, high_h = 91, 103  # Example range for yellow hue
#     low_s, high_s = 200, 255 # Example range for saturation
#     low_v, high_v = 140, 255 # Example range for value
    low_h, high_h = low_h_slider.value, high_h_slider.value
    low_s, high_s = low_s_slider.value, high_s_slider.value
    low_v, high_v = low_v_slider.value, high_v_slider.value

    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define the lower and upper bounds of the HSV threshold
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])
    
    # Create a mask where pixels within the threshold are white, and others are black
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Create an all black image
    black_image = np.zeros_like(image)
    
    # Copy the pixels from the original image where the mask is white
    result_image = np.where(mask[:, :, None] == 255, image, black_image)
    
    return result_image



In [ ]:
def update_image(camera_image):
    global x, y, r, lock
    # 画像処理ロジックをここに追加
    image = np.copy(camera_image)
    processed_frame = apply_hsv_threshold(image)
    # ... その他の画像処理
    
    global model,device
    with lock:
        # ロックを取得してからグローバル変数を更新
        x, y, r = model_input(processed_frame, model, device)
        x = int(x * image.shape[1])
        y = int(y * image.shape[0])
        r = int(r * (image.shape[1] * 1.414))
        if r < 0:
            r = 0
            
    processed_image = cv2.circle(processed_frame, (x, y), r, (0, 255, 0), 2)
    
    # processed_frameに基づいてimg_with_circleを更新
    jpeg_img = bgr8_to_jpeg(processed_image)
    # processed_frameに基づいてimg_with_circleを更新
    return jpeg_img

traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=update_image)

In [ ]:
left_layout = widgets.VBox([image_layout,slider])
right_layout = widgets.VBox([robot_slider])
layout_ui = widgets.HBox([left_layout, right_layout])
display(layout_ui)

In [ ]:
robot = Robot()
move = [] #[[左,右],...]

GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
GPIO.setup(31,GPIO.IN) #トグルスイッチ左
GPIO.setup(33,GPIO.OUT) #トグルスイッチ中，ボタンHIGH側
GPIO.setup(35,GPIO.IN) #トグルスイッチ右
GPIO.setup(37,GPIO.IN) #ボタンLOW側

GPIO.output(33,GPIO.HIGH) #HIGH出力

def sigmoid(x, k=0.06, x0=112):
    print("sigmoid")
    if x==0:
        return 0
    else:
        return 2 / (1 + np.exp(-k * (x - x0)))

def set_speed(r):
    print("set speed")
    #はやさの判定
    if r<far_slider.value:
        speed = 0.5
    elif far_slider.value<r and r<near_slider.value:
        speed = 0.2
    elif near_slider.value<r:
        speed = 0
        
    return speed

def motor_control(x, y, r):
    print("control")
    v = (x - 112) / 112
    scale = sigmoid(abs(v))
    
    # speed = speed_slider.value
    speed = set_speed(r)
    left_speed = speed + v*speed*scale #左モーター出力
    right_speed = speed - v*speed*scale #右モーター出力
    robot.set_motors(left_speed,right_speed) #左右のモーター出力
    move.append([left_speed,right_speed]) #配列に記録(後ろに追加)
    
    
def motor_control_thread():
    global x, y, r, lock

    try:
        while True:
            toggle0 = GPIO.input(31) # トグルスイッチ左向き（記録モード）
            toggle1 = GPIO.input(35) # トグルスイッチ右向き（実行モード）
            button = GPIO.input(37) # 実行ボタンの入力を受け取る

            if toggle0 == 0:#記録モード
                with lock:
                    # ロックを取得してからグローバル変数を読み取る
                    local_x, local_y, local_r = x, y, r
                print("loop")
                motor_control(local_x, local_y, local_r)
                interval = interval_slider.value #間隔
                time.sleep(interval)
#                 if toggle0 == GPIO.LOW:
#                     break
                
            elif toggle1 == GPIO.HIGH:  # 実行モード
                before = button
                now = button
                if now == GPIO.HIGH and before == GPIO.LOW:  # LOW→HIGH（押した瞬間）を検知
                    for i in move:
                        robot.set_motors(move[i][0], move[i][1])
                        time.sleep(interval)

                before = now  # 前の状態を更新

    except Exception as e:
        print(f"エラーが発生しました: {e}")

    finally:
        # 例外が発生した場合も、モータを停止させる
        robot.stop()
        GPIO.cleanup()


